# Breakfast Restaurant Locations
The aim of this notebook is to determine which communities in Calgary are best suited for a newly opening breakfast restaurant.

In [1]:
# Import libs
import pandas as pd
import numpy as np
import plotly.offline as py # TODO switch to online mode to generate iframe-ready graphs on plot.ly
import plotly.graph_objs as go
import folium
import json, requests

Let's start by gathering the 2016 census data, which is already grouped into communities. We need to use data from 2016, as this was the most recent year that demographic information was collected. The next census with demographics will be later this year (2019). Additionally, since all Open Calgary data is available via the SODA API, we can select columns in our request (a bit like SQL). 

In [4]:
request_fields = ['CLASS', 'CLASS_CODE', 'COMM_CODE', 'NAME', 'COMM_STRUCTURE', 'RES_CNT', 'DWELL_CNT', 'EMPLYD_CNT',
                  'MF_0_4', 'MF_5_14', 'MF_15_19', 'MF_20_24', 'MF_25_34', 'MF_35_44', 'MF_45_54', 'MF_55_64', 'MF_65_74', 'MF_75']
url = 'https://data.calgary.ca/resource/mndt-3a4i.csv?$select=' + ','.join(request_fields) # Census 2016
print(url)
census = pd.read_csv(url)
census.head()

https://data.calgary.ca/resource/mndt-3a4i.csv?$select=CLASS,CLASS_CODE,COMM_CODE,NAME,COMM_STRUCTURE,RES_CNT,DWELL_CNT,EMPLYD_CNT,MF_0_4,MF_5_14,MF_15_19,MF_20_24,MF_25_34,MF_35_44,MF_45_54,MF_55_64,MF_65_74,MF_75


,CLASS,CLASS_CODE,COMM_CODE,COMM_STRUCTURE,DWELL_CNT,EMPLYD_CNT,MF_0_4,MF_15_19,MF_20_24,MF_25_34,MF_35_44,MF_45_54,MF_55_64,MF_5_14,MF_65_74,MF_75,NAME,RES_CNT
0,Industrial,2,MNI,EMPLOYMENT,3,0,0,0,0,0,0,0,0,0,0,0,MANCHESTER INDUSTRIAL,0
1,Industrial,2,EFV,EMPLOYMENT,1,0,0,0,1,4,4,4,2,0,0,0,EAST FAIRVIEW INDUSTRIAL,15
2,Major Park,3,COP,OTHER,0,0,0,0,0,0,0,0,0,0,0,0,CANADA OLYMPIC PARK,0
3,Industrial,2,FHI,EMPLOYMENT,1,0,0,29,36,62,115,30,20,0,12,0,FOOTHILLS,304
4,Industrial,2,SHI,EMPLOYMENT,150,138,15,9,7,30,31,47,65,9,37,23,SHEPARD INDUSTRIAL,273


We also want geodata on every community. `calgary_geo` is a geojson file defining the boundries of all communities ([downloaded from here](https://data.calgary.ca/Base-Maps/Community-Boundaries/ab7m-fwn6)). `comm_locs` is the centroids of all communities, from [this dataset](https://data.calgary.ca/Base-Maps/Community-Points/j9ps-fyst).

In [7]:
calgary_geo = 'data/Community Boundaries.geojson' # This should be identical to the multipolygon field included in the census, but it saves me some hassle of transforming data for Folium's picky needs.
request_fields = ['COMM_CODE', 'latitude', 'longitude']
url = 'https://data.calgary.ca/resource/kzbm-mn66.csv?$select=' + ','.join(request_fields) # 
comm_locs = pd.read_csv(url)
comm_locs.head()

,COMM_CODE,latitude,longitude
0,EYA,51.012993,-114.073623
1,ESH,50.951252,-113.950957
2,PEN,51.045863,-113.947522
3,SCE,51.112684,-114.217279
4,RAN,51.119163,-114.183601


In [14]:
census = census.merge(comm_locs, on='COMM_CODE')
census.head()

,CLASS,CLASS_CODE,COMM_CODE,COMM_STRUCTURE,DWELL_CNT,EMPLYD_CNT,MF_0_4,MF_15_19,MF_20_24,MF_25_34,MF_35_44,MF_45_54,MF_55_64,MF_5_14,MF_65_74,MF_75,NAME,RES_CNT,latitude,longitude
0,Industrial,2,MNI,EMPLOYMENT,3,0,0,0,0,0,0,0,0,0,0,0,MANCHESTER INDUSTRIAL,0,51.013263,-114.057269
1,Industrial,2,EFV,EMPLOYMENT,1,0,0,0,1,4,4,4,2,0,0,0,EAST FAIRVIEW INDUSTRIAL,15,50.984817,-114.037243
2,Major Park,3,COP,OTHER,0,0,0,0,0,0,0,0,0,0,0,0,CANADA OLYMPIC PARK,0,51.080787,-114.217426
3,Industrial,2,FHI,EMPLOYMENT,1,0,0,29,36,62,115,30,20,0,12,0,FOOTHILLS,304,50.994166,-113.981982
4,Industrial,2,SHI,EMPLOYMENT,150,138,15,9,7,30,31,47,65,9,37,23,SHEPARD INDUSTRIAL,273,50.961480,-113.992775
5,Major Park,3,FPK,PARKS,0,0,0,0,0,0,0,0,0,0,0,0,FISH CREEK PARK,0,50.909602,-114.026521
6,Residual Sub Area,4,12B,UNDEVELOPED,7,9,1,0,0,1,5,0,2,3,2,3,12B,17,50.939231,-113.898687
7,Industrial,2,ST4,EMPLOYMENT,4,0,0,0,0,0,0,0,0,0,0,0,STONEY 4,0,51.176204,-114.002762
8,Industrial,2,ST1,EMPLOYMENT,4,0,0,0,0,0,0,0,0,0,0,0,STONEY 1,0,51.161184,-114.024717
9,Industrial,2,NAW,EMPLOYMENT,3,0,0,0,0,0,0,0,0,0,0,0,NORTH AIRWAYS,0,51.087140,-114.010880


Let's quickly generate a map of Calgary to verify that our geojson matches our dataframe. Unfortunately, complex maps do not work natively in jupyterlab, so I have to use the workaround [here](https://github.com/python-visualization/folium/issues/812).

In [15]:
def embed_map(m, name):
    from IPython.display import IFrame

    m.save(name+'.html')
    return IFrame(name+'.html', width='100%', height='750px')

In [18]:
map_communities = folium.Map(location=[51.0486, -114.0708], zoom_start=11)

folium.Choropleth(
    geo_data = calgary_geo,
    data = census,
    columns = ['NAME', 'RES_CNT'],
    key_on = 'feature.properties.name',
    fill_color='YlGn'
).add_to(map_communities)
map_communities.save('comm.html')
embed_map(map_communities, 'comms')

The black communities are expected, as they have been created between 2016 and 2018, and therefore have no census data from 2016.

Next, let's get all the breakfast restaurants within the bounding box of each community:

In [88]:
def get_bounding_box(multipoly):
    coord_list = multipoly[16:-3].split(',')
    sw_long, sw_lat = list(map(float, coord_list[0].split(' ')))
    ne_lat, ne_long = sw_lat, sw_long
    for i in range(1,len(coord_list)):
        lat_long = filter(None, coord_list[i].split(' '))
        long_new, lat_new = list(map(float, lat_long))
        sw_lat = max(sw_lat, lat_new)
        sw_long = min(sw_long, long_new)
        ne_lat = min(ne_lat, lat_new)
        ne_long = max(ne_long, long_new)
    return (str(round(sw_lat, 8))+','+str(round(sw_long, 8)), str(round(ne_lat, 8))+','+str(round(ne_long, 8)))
    
get_bounding_box(census['multipolygon'][0])

('50.89350838,-114.05439198', '50.87569155,-114.00181285')

In [89]:
all_venues = []

for index, row in census.iterrows():
    bounding_box = get_bounding_box(row['multipolygon'])
    params = dict(
        client_id=client_id, # These have been defined in the iPython terminal at runtime to keep them out of the notebook.
        client_secret=client_secret,
        v='20180323',
        # near='Calgary, Alberta',
        limit=50,
        sw = bounding_box[0],
        ne = bounding_box[1],
        query='breakfast'
    )
    res = requests.get(url='https://api.foursquare.com/v2/venues/explore', params=params).text
    print('---')
    print(bounding_box[0])
    venues = json.loads(res)['response']['groups'][0]['items']
    print(json.loads(res))
    for venue in venues:
        venue_row = [
            row['NAME'],
            row['COMM_CODE'],
            venue['venue']['name'],
            venue['venue']['location']['lat'],
            venue['venue']['location']['lng'],
            venue['categories']
        ]
        print(venue_row)
        all_venues.append(venue_row)


venue_df = pd.DataFrame(all_venues, columns = ['Community Name', 'Community Code', 'Venue Name', 'Lat', 'Long', 'Categories'])

---
50.89350838,-114.05439198
{'response': {'groups': [{'items': [], 'name': 'recommended', 'type': 'Recommended Places'}], 'query': 'breakfast', 'headerFullLocation': 'Chaparral, Calgary', 'headerLocationGranularity': 'neighborhood', 'warning': {'text': 'There aren\'t a lot of results for "breakfast." Try something more general, reset your filters, or expand the search area.'}, 'totalResults': 0, 'headerLocation': 'Chaparral', 'suggestedBounds': {'ne': {'lat': 50.87569155, 'lng': -114.00181285}, 'sw': {'lat': 50.89350838, 'lng': -114.05439198}}}, 'meta': {'code': 200, 'requestId': '5c4a3f684c1f671d0005f96d'}}
---
51.04722243,-113.94989036
{'response': {'groups': [{'items': [], 'name': 'recommended', 'type': 'Recommended Places'}], 'query': 'breakfast', 'headerFullLocation': 'Penbrooke Meadows, Calgary', 'headerLocationGranularity': 'neighborhood', 'warning': {'text': 'There aren\'t a lot of results for "breakfast." Try something more general, reset your filters, or expand the search a

KeyboardInterrupt: 